In [ ]:
# import data from excel into pandas
import pandas as pd

# read data from excel, from sheet 'Ratings'
df = pd.read_excel('GroupMemberEvaluation.xlsx', sheet_name='Ratings')
df_student_ratings = df.iloc[:,5:].drop('The rated work of student id', axis=1) # Only take rating columns and student name column
df_group_ratings   = df.iloc[:,4:].drop(['The rated work of student id', 'The rated work of student name'], axis=1) # Only take rating columns and group column

#### METRIC 1: Average rating and std of each student ####

# Group by "The rated work of student name" and calculate the mean and std of the ratings
df_student_average = df_student_ratings.groupby('The rated work of student name').mean()
df_student_std     = df_student_ratings.groupby('The rated work of student name').std()

In [ ]:
#### METRIC 2: Students compared to their group ####

# Group by "The rated work of group" and calculate the mean of the ratings
df_group_average = df_group_ratings.groupby('The rated work of group').mean()

# Add group column by merging with the original dataframe
interm_student_average_merge = pd.merge(df_student_average, df[['The rated work of student name', 'The rated work of group']].drop_duplicates(), on='The rated work of student name', how='inner')
# Merge student average with group average
df_comparison = pd.merge(interm_student_average_merge, df_group_average, on='The rated work of group', how='inner')
# Subtract group average from student average
df_comparison.iloc[:,1:15] = df_comparison.iloc[:,1:15].values - df_comparison.iloc[:,16:].values
# Calculate mean of the differences
df_comparison['Mean difference from group'] = df_comparison.iloc[:,1:15].mean(axis=1)

In [ ]:
#### METRIC 3: Summary table and complete tables ####

df_summary = df_comparison[['The rated work of student name', 'The rated work of group', 'Mean difference from group']]
# Add student average and std
df_student_average["Average"] = df_student_average.iloc[:,1:].mean(axis=1)
df_student_std["Average Std"] = df_student_std.iloc[:,1:].mean(axis=1)
df_group_average["Average"] = df_group_average.iloc[:,1:].mean(axis=1)
df_summary = pd.merge(df_summary, df_student_average[['Average']], left_on='The rated work of student name', right_index=True, how='inner')
df_summary = pd.merge(df_summary, df_student_std[['Average Std']], left_on='The rated work of student name', right_index=True, how='inner')

In [ ]:
#### METRIC 4: Difference between average grade given and received ####
# As function: identifier ratings given, identifier ratings received, rating columns, full dataframe

def calculate_difference(given_identifier, received_identifier, rating_column_names, df):
    
    # Filter out the rating columns
    df_given = df[[given_identifier] + rating_column_names]
    df_received = df[[received_identifier] + rating_column_names]

    # Calculate the mean of the ratings given
    df_given_mean = df_given.groupby(given_identifier).mean().reset_index()

    # Calculate the mean of the ratings received
    df_received_mean = df_received.groupby(received_identifier).mean().reset_index()

    df_merged = pd.merge(df_given_mean, df_received_mean, how='outer', 
                        left_on=given_identifier, right_on=received_identifier)

    # Subtract corresponding columns
    for name in rating_column_names:
        df_merged[name + '_difference'] = df_merged[name + '_x'] - df_merged[name + '_y']

    # Average the differences
    df_merged['Given-received mean difference'] = df_merged[[name + '_difference' for name in rating_column_names]].mean(axis=1)

    # Check for the longer dataframe, use as index
    longer =  given_identifier if len(df_given_mean) > len(df_received_mean) else received_identifier
    df_output = df_merged[[longer, 'Given-received mean difference']].set_index(longer)

    return df_output

rating_column_names = ['Targeted work', 'Team orientation ',
       'Efficiency in work habits', 'Work technique', 'Guidance',
       'Independence', 'Cooperation', 'Conflict/criticism',
       'Willingness to take on responsibility', 'Flexibility',
       'Situation-appropriate behavior', 'Communication skills', 'Motivation',
       ' Resilience/Ability to work under pressure']
given_identifier = 'Student name'
received_identifier = 'The rated work of student name'


df_output = calculate_difference(given_identifier, received_identifier, rating_column_names, df)

# Add output to summary table
df_summary = pd.merge(df_summary, df_output, left_on='The rated work of student name', right_index=True, how='inner')